In [6]:
import tensorflow as tf
import os
import json
import pandas as pd
import re
import numpy as np
import time
import matplotlib.pyplot as plt
import collections
import random
import requests
from math import sqrt
from PIL import Image
from tqdm.auto import tqdm

In [1]:
if not os.path.exists('/content/data/'):

    api_token = {"username": "<-- your username -->",
                 "key": "<-- your api key -->"}

    with open('/content/kaggle.json', 'w') as file:
        json.dump(api_token, file)

    os.environ["KAGGLE_CONFIG_DIR"] = "/content/"

    os.system('kaggle datasets download -d adityajn105/flickr8k')
    os.makedirs('/content/data/', exist_ok=True)
    os.system('mv /content/flickr8k.zip /content/data/flickr8k.zip')
    os.system('unzip -q /content/data/flickr8k.zip -d /content/data/')
    os.remove('/content/data/flickr8k.zip')
'''在kaggle上下载flickr8k数据集，8000张图，建议只挑很小的一部分来训练'''

FileNotFoundError: [Errno 2] No such file or directory: '/content/kaggle.json'

In [ ]:
import pandas as pd

# 从 '/content/data/captions.txt' 文件中读取字幕数据
captions = pd.read_csv('/content/data/captions.txt')

# 将 'image' 列的值进行处理，拼接路径 '/content/data/Images/' 和文件名
captions['image'] = captions['image'].apply(lambda x: f'/content/data/Images/{x}')

# 打印前几行数据
captions.head()

In [2]:
import re

def preprocess(text):
    # 将文本转换为小写
    text = text.lower()
    
    # 去除非字母和非空格字符
    text = re.sub(r'[^\w\s]', '', text)
    
    # 将连续的空格替换为单个空格
    text = re.sub('\s+', ' ', text)
    
    # 去除文本两端的空格
    text = text.strip()
    
    # 在文本前后添加特殊标记
    text = '[start] ' + text + ' [end]'
    
    return text

In [3]:
captions['caption'] = captions['caption'].apply(preprocess)
captions.head()
# 注释：

# 使用 apply() 方法将 preprocess 函数应用到 captions['caption'] 列的每个元素上，对每个字幕进行预处理。
# 将预处理后的字幕存储回 captions['caption'] 列。
# 使用 head() 方法显示前几行的数据。

NameError: name 'captions' is not defined

In [ ]:
random_row = captions.sample(1).iloc[0]
print(random_row.caption)
print()
im = Image.open(random_row.image)

# 注释：

# 使用 sample() 方法从 captions 数据框中随机抽取一行数据，并通过 iloc[0] 获取抽取行的内容。
# 打印随机抽取的字幕内容。
# 创建 Image 对象，使用 Image.open() 方法打开对应的图像文件。
# 显示图像。

In [4]:
MAX_LENGTH = 40
VOCABULARY_SIZE = 10000
BATCH_SIZE = 32
BUFFER_SIZE = 1000
EMBEDDING_DIM = 512
UNITS = 512
'''MAX_LENGTH：定义生成的字幕的最大长度，超过该长度的字幕将被截断或省略。
VOCABULARY_SIZE：定义字幕词汇表的大小，即词汇表中不同单词的数量。
BATCH_SIZE：定义用于训练的每个批次的样本数量。
BUFFER_SIZE：定义用于数据集缓冲的大小，用于混洗和预取数据。
EMBEDDING_DIM：定义嵌入层的维度，用于将单词编码为密集向量表示。
UNITS：定义LSTM或Transformer模型中隐藏层的单元数量。这决定了模型的复杂度和记忆能力。'''

'MAX_LENGTH：定义生成的字幕的最大长度，超过该长度的字幕将被截断或省略。\nVOCABULARY_SIZE：定义字幕词汇表的大小，即词汇表中不同单词的数量。\nBATCH_SIZE：定义用于训练的每个批次的样本数量。\nBUFFER_SIZE：定义用于数据集缓冲的大小，用于混洗和预取数据。\nEMBEDDING_DIM：定义嵌入层的维度，用于将单词编码为密集向量表示。\nUNITS：定义LSTM或Transformer模型中隐藏层的单元数量。这决定了模型的复杂度和记忆能力。'

In [5]:
# 创建文本标记器
tokenizer = tf.keras.layers.TextVectorization(
    max_tokens=VOCABULARY_SIZE,  # 定义词汇表大小
    standardize=None,  # 不应用标准化函数
    output_sequence_length=MAX_LENGTH  # 定义输出序列长度
)

# 适应标记器到字幕数据
tokenizer.adapt(captions['caption'])  # 将标记器适应到字幕数据中的'caption'列

NameError: name 'tf' is not defined

In [7]:
# 创建StringLookup层用于单词到索引的映射
word2idx = tf.keras.layers.StringLookup(
    mask_token="",  # 设置掩码标记为空
    vocabulary=tokenizer.get_vocabulary()  # 使用之前创建的tokenizer的词汇表作为单词列表
)

# 创建StringLookup层用于索引到单词的映射
idx2word = tf.keras.layers.StringLookup(
    mask_token="",  # 设置掩码标记为空
    vocabulary=tokenizer.get_vocabulary(),  # 使用之前创建的tokenizer的词汇表作为单词列表
    invert=True  # 反转映射，从索引到单词
)

NameError: name 'tokenizer' is not defined

In [ ]:
# 创建一个字典，用于存储每个图像对应的多个标题
img_to_cap_vector = collections.defaultdict(list)

# 将图像和标题一一对应，存入字典中
for img, cap in zip(captions['image'], captions['caption']):
    img_to_cap_vector[img].append(cap)

# 获取图像的键列表并进行随机打乱
img_keys = list(img_to_cap_vector.keys())
random.shuffle(img_keys)

# 将图像键列表切分为训练集和验证集
slice_index = int(len(img_keys) * 0.8)
img_name_train_keys, img_name_val_keys = (img_keys[:slice_index], 
                                          img_keys[slice_index:])

# 构建训练集的图像和标题列表
train_imgs = []
train_captions = []
for imgt in img_name_train_keys:
    capt_len = len(img_to_cap_vector[imgt])
    train_imgs.extend([imgt] * capt_len)
    train_captions.extend(img_to_cap_vector[imgt])

# 构建验证集的图像和标题列表
val_imgs = []
val_captions = []
for imgv in img_name_val_keys:
    capv_len = len(img_to_cap_vector[imgv])
    val_imgs.extend([imgv] * capv_len)
    val_captions.extend(img_to_cap_vector[imgv])

In [ ]:
def load_data(img_path, caption):
    # 从文件路径中读取图像数据
    img = tf.io.read_file(img_path)
    # 解码图像数据为张量
    img = tf.io.decode_jpeg(img, channels=3)
    # 调整图像大小为指定尺寸
    img = tf.keras.layers.Resizing(299, 299)(img)
    # 归一化图像数据
    img = img / 255.
    # 使用预先定义的tokenizer对标题进行编码
    caption = tokenizer(caption)
    return img, caption

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_imgs, train_captions))

# 使用map函数对训练数据集进行预处理，同时设置并行调用的数量
train_dataset = train_dataset.map(load_data, num_parallel_calls=tf.data.AUTOTUNE)

# 打乱数据并设置批次大小
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

val_dataset = tf.data.Dataset.from_tensor_slices((val_imgs, val_captions))

# 使用map函数对验证数据集进行预处理，同时设置并行调用的数量
val_dataset = val_dataset.map(load_data, num_parallel_calls=tf.data.AUTOTUNE)

# 打乱数据并设置批次大小
val_dataset = val_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

In [ ]:
image_augmentation = tf.keras.Sequential(
    [
        tf.keras.layers.RandomFlip("horizontal"),
        tf.keras.layers.RandomRotation(0.2),
        tf.keras.layers.RandomContrast(0.3),
    ]
)
'''
这段代码定义了一个图像增强的序列模型，其中包含三个图像增强的层：

tf.keras.layers.RandomFlip("horizontal")：随机水平翻转图像。
tf.keras.layers.RandomRotation(0.2)：随机旋转图像，旋转角度在-0.2到0.2之间。
tf.keras.layers.RandomContrast(0.3)：随机调整图像对比度，对比度系数在0.7到1.3之间。
你可以将这个图像增强的序列模型应用于训练数据集的图像上，以增加数据的多样性和鲁棒性。
例如，可以在load_data函数中的图像处理部分之前使用该模型，将图像增强应用到每个训练样本上。
这样可以在训练过程中随机改变图像的水平翻转、旋转和对比度，从而增加模型对不同变体的图像的泛化能力。'''

In [ ]:
def CNN_Encoder():
    # 使用预训练的InceptionV3模型作为基础
    inception_v3 = tf.keras.applications.InceptionV3(
        include_top=False,
        weights='imagenet'
    )
    # 冻结InceptionV3模型的权重，不参与训练
    inception_v3.trainable = False

    # 获取InceptionV3模型的输出层
    output = inception_v3.output
    # 重塑输出，转换为2D张量
    output = tf.keras.layers.Reshape(
        (-1, output.shape[-1]))(output)

    # 创建CNN编码器模型，输入为InceptionV3模型的输入层，输出为重塑后的特征张量
    cnn_model = tf.keras.models.Model(inception_v3.input, output)
    return cnn_model


这段代码定义了一个CNN编码器模型，使用预训练的InceptionV3模型作为基础。

首先，通过tf.keras.applications.InceptionV3函数创建了一个InceptionV3模型，设置include_top=False表示不包括顶部的全连接层，weights='imagenet'表示加载在ImageNet上预训练的权重。

接下来，将InceptionV3模型的输出层作为输入，并通过tf.keras.layers.Reshape层将输出进行重塑，将其转换为形状为(-1, output.shape[-1])的张量。这样做是为了将输出转换为2D张量，便于后续的处理。

最后，通过tf.keras.models.Model函数创建一个模型，将InceptionV3模型的输入层和重塑后的输出层作为输入和输出，得到最终的CNN编码器模型。

这个CNN编码器模型将输入的图像通过预训练的InceptionV3模型进行特征提取，并将提取到的特征转换为2D张量作为输出。这些特征将用作后续的图像字幕生成模型的输入。

In [ ]:
class TransformerEncoderLayer(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads):
        super().__init__()
        self.layer_norm_1 = tf.keras.layers.LayerNormalization()
        self.layer_norm_2 = tf.keras.layers.LayerNormalization()
        self.attention = tf.keras.layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense = tf.keras.layers.Dense(embed_dim, activation="relu")
    
    def call(self, x, training):
        # 应用 Layer Normalization
        x = self.layer_norm_1(x)
        # 进行全连接层操作
        x = self.dense(x)

        # 进行多头自注意力机制
        attn_output = self.attention(
            query=x,
            value=x,
            key=x,
            attention_mask=None,
            training=training
        )

        # 跳连接和残差连接
        x = self.layer_norm_2(x + attn_output)
        return x
    '''这段代码定义了Transformer模型的一个编码器层。在__init__方法中，
    使用tf.keras.layers.LayerNormalization定义了两个Layer Normalization层，
    分别用于在自注意力机制前后进行归一化操作。使用tf.keras.layers.MultiHeadAttention定义了一个多头注意力层，
    其中num_heads表示头的数量，key_dim表示注意力机制中键的维度。使用tf.keras.layers.Dense定义了一个全连接层，
    将输入的维度转换为embed_dim，并使用ReLU作为激活函数。

在call方法中，首先对输入应用Layer Normalization进行归一化操作，
然后通过全连接层进行维度转换。接下来，调用多头注意力层，传入输入的查询、键和值，并指定注意力屏蔽和训练模式。
注意力层会计算注意力权重，并将其应用于值，得到注意力输出。最后，将注意力输出与跳连接的输入相加，并再次应用Layer Normalization
，得到编码器层的输出。这个编码器层可以被堆叠多次以构建更深的Transformer编码器。'''

In [ ]:
class Embeddings(tf.keras.layers.Layer):
    def __init__(self, vocab_size, embed_dim, max_len):
        super().__init__()
        self.token_embeddings = tf.keras.layers.Embedding(
            vocab_size, embed_dim)
        self.position_embeddings = tf.keras.layers.Embedding(
            max_len, embed_dim, input_shape=(None, max_len))
    
    def call(self, input_ids):
        # 获取输入序列的长度
        length = tf.shape(input_ids)[-1]
        # 生成位置编码的索引
        position_ids = tf.range(start=0, limit=length, delta=1)
        position_ids = tf.expand_dims(position_ids, axis=0)

        # 获取词嵌入和位置嵌入
        token_embeddings = self.token_embeddings(input_ids)
        position_embeddings = self.position_embeddings(position_ids)

        # 将词嵌入和位置嵌入相加得到最终的嵌入向量
        return token_embeddings + position_embeddings


这段代码定义了Transformer模型的嵌入层。在__init__方法中，使用tf.keras.layers.Embedding定义了两个嵌入层，分别用于词嵌入和位置嵌入。vocab_size表示词汇表的大小，embed_dim表示嵌入的维度，max_len表示输入序列的最大长度。

在call方法中，首先通过tf.shape获取输入序列的长度，然后使用tf.range生成位置编码的索引。将位置编码的索引进行扩展，使其与输入序列的维度相匹配。

接下来，通过词嵌入层将输入序列转换为词嵌入向量，通过位置嵌入层将位置编码转换为位置嵌入向量。

最后，将词嵌入向量和位置嵌入向量相加得到最终的嵌入向量。这个嵌入向量将作为输入传递给Transformer的编码器层。

In [ ]:
Embeddings(tokenizer.vocabulary_size(), EMBEDDING_DIM, MAX_LENGTH)(next(iter(train_dataset))[1]).shape

这段代码计算了使用Embeddings类进行嵌入操作后的输出形状。首先创建了Embeddings对象，传入了词汇表大小、嵌入维度和最大序列长度作为参数。

然后通过next(iter(train_dataset))[1]获取训练数据集中的一个批次的输入序列，并将其作为参数传递给嵌入层的call方法。得到的输出被赋值给embedding_output。

最后打印出embedding_output的形状，即嵌入后的向量形状。

In [ ]:
class TransformerDecoderLayer(tf.keras.layers.Layer):

    def __init__(self, embed_dim, units, num_heads):
        super().__init__()
        self.embedding = Embeddings(
            tokenizer.vocabulary_size(), embed_dim, MAX_LENGTH)

        self.attention_1 = tf.keras.layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim, dropout=0.1
        )
        self.attention_2 = tf.keras.layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim, dropout=0.1
        )

        self.layernorm_1 = tf.keras.layers.LayerNormalization()
        self.layernorm_2 = tf.keras.layers.LayerNormalization()
        self.layernorm_3 = tf.keras.layers.LayerNormalization()

        self.ffn_layer_1 = tf.keras.layers.Dense(units, activation="relu")
        self.ffn_layer_2 = tf.keras.layers.Dense(embed_dim)

        self.out = tf.keras.layers.Dense(tokenizer.vocabulary_size(), activation="softmax")

        self.dropout_1 = tf.keras.layers.Dropout(0.3)
        self.dropout_2 = tf.keras.layers.Dropout(0.5)
    

    def call(self, input_ids, encoder_output, training, mask=None):
        embeddings = self.embedding(input_ids)

        combined_mask = None
        padding_mask = None
        
        if mask is not None:
            causal_mask = self.get_causal_attention_mask(embeddings)
            padding_mask = tf.cast(mask[:, :, tf.newaxis], dtype=tf.int32)
            combined_mask = tf.cast(mask[:, tf.newaxis, :], dtype=tf.int32)
            combined_mask = tf.minimum(combined_mask, causal_mask)

        attn_output_1 = self.attention_1(
            query=embeddings,
            value=embeddings,
            key=embeddings,
            attention_mask=combined_mask,
            training=training
        )

        out_1 = self.layernorm_1(embeddings + attn_output_1)

        attn_output_2 = self.attention_2(
            query=out_1,
            value=encoder_output,
            key=encoder_output,
            attention_mask=padding_mask,
            training=training
        )

        out_2 = self.layernorm_2(out_1 + attn_output_2)

        ffn_out = self.ffn_layer_1(out_2)
        ffn_out = self.dropout_1(ffn_out, training=training)
        ffn_out = self.ffn_layer_2(ffn_out)

        ffn_out = self.layernorm_3(ffn_out + out_2)
        ffn_out = self.dropout_2(ffn_out, training=training)
        preds = self.out(ffn_out)
        return preds


    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)],
            axis=0
        )
        return tf.tile(mask, mult)

这段代码定义了一个Transformer解码器层的自定义Keras层，包括了嵌入层、多头注意力机制、层归一化、前馈神经网络等组件。

在call方法中，首先将输入序列input_ids通过嵌入层self.embedding进行嵌入操作。然后根据是否存在掩码mask，构造综合注意力掩码和填充掩码。利用多头注意力机制self.attention_1和self.attention_2分别进行自注意力和编码器-解码器注意力的计算。

接着通过层归一化和残差连接将注意力输出与嵌入层的输出进行结合，并经过前馈神经网络和层归一化进行进一步的处理。

最后，使用self.out进行分类预测，并返回预测结果。

get_causal_attention_mask方法用于生成自回归式的注意力掩码，保证解码器在生成序列时只能依赖于已生成的部分，不会使用后续的信息。

In [ ]:
class ImageCaptioningModel(tf.keras.Model):

    def __init__(self, cnn_model, encoder, decoder, image_aug=None):
        super().__init__()
        self.cnn_model = cnn_model
        self.encoder = encoder
        self.decoder = decoder
        self.image_aug = image_aug
        self.loss_tracker = tf.keras.metrics.Mean(name="loss")
        self.acc_tracker = tf.keras.metrics.Mean(name="accuracy")


    def calculate_loss(self, y_true, y_pred, mask):
        loss = self.loss(y_true, y_pred)
        mask = tf.cast(mask, dtype=loss.dtype)
        loss *= mask
        return tf.reduce_sum(loss) / tf.reduce_sum(mask)


    def calculate_accuracy(self, y_true, y_pred, mask):
        accuracy = tf.equal(y_true, tf.argmax(y_pred, axis=2))
        accuracy = tf.math.logical_and(mask, accuracy)
        accuracy = tf.cast(accuracy, dtype=tf.float32)
        mask = tf.cast(mask, dtype=tf.float32)
        return tf.reduce_sum(accuracy) / tf.reduce_sum(mask)
    

    def compute_loss_and_acc(self, img_embed, captions, training=True):
        encoder_output = self.encoder(img_embed, training=True)
        y_input = captions[:, :-1]
        y_true = captions[:, 1:]
        mask = (y_true != 0)
        y_pred = self.decoder(
            y_input, encoder_output, training=True, mask=mask
        )
        loss = self.calculate_loss(y_true, y_pred, mask)
        acc = self.calculate_accuracy(y_true, y_pred, mask)
        return loss, acc

    
    def train_step(self, batch):
        imgs, captions = batch

        if self.image_aug:
            imgs = self.image_aug(imgs)
        
        img_embed = self.cnn_model(imgs)

        with tf.GradientTape() as tape:
            loss, acc = self.compute_loss_and_acc(
                img_embed, captions
            )
    
        train_vars = (
            self.encoder.trainable_variables + self.decoder.trainable_variables
        )
        grads = tape.gradient(loss, train_vars)
        self.optimizer.apply_gradients(zip(grads, train_vars))
        self.loss_tracker.update_state(loss)
        self.acc_tracker.update_state(acc)

        return {"loss": self.loss_tracker.result(), "acc": self.acc_tracker.result()}
    

    def test_step(self, batch):
        imgs, captions = batch

        img_embed = self.cnn_model(imgs)

        loss, acc = self.compute_loss_and_acc(
            img_embed, captions, training=False
        )

        self.loss_tracker.update_state(loss)
        self.acc_tracker.update_state(acc)

        return {"loss": self.loss_tracker.result(), "acc": self.acc_tracker.result()}

    @property
    def metrics(self):
        return [self.loss_tracker, self.acc_tracker]

这是一个用于图像字幕生成的模型类ImageCaptioningModel，继承自tf.keras.Model。

在__init__方法中，初始化了CNN模型cnn_model、编码器encoder、解码器decoder和图像增强器image_aug。同时，定义了用于跟踪损失和准确率的loss_tracker和acc_tracker。

calculate_loss方法计算损失函数，根据预测值y_pred和真实值y_true以及掩码mask计算交叉熵损失，并根据掩码对损失进行加权。

calculate_accuracy方法计算准确率，根据预测值y_pred和真实值y_true以及掩码mask计算序列准确率，并根据掩码对准确率进行加权。

compute_loss_and_acc方法用于计算损失和准确率。首先通过编码器self.encoder对图像进行编码得到特征向量encoder_output，然后根据输入序列captions预测下一个词，并计算损失和准确率。

train_step方法定义了每个训练步骤的操作。首先对图像进行图像增强处理（如果有的话），然后通过CNN模型self.cnn_model对图像进行特征提取得到图像嵌入img_embed。接着使用自定义的compute_loss_and_acc方法计算损失和准确率，并使用梯度带计算梯度并更新模型参数。最后更新损失和准确率的跟踪器。

test_step方法定义了每个测试步骤的操作。通过CNN模型对图像进行特征提取得到图像嵌入img_embed，然后使用compute_loss_and_acc方法计算损失和准确率。最后更新损失和准确率的跟踪器。

metrics属性返回模型的度量指标，包括损失和准确率的跟踪器。

这个模型类可以用于训练和评估图像字幕生成模型。

In [ ]:
encoder = TransformerEncoderLayer(EMBEDDING_DIM, 1)  # 创建Transformer编码器层
decoder = TransformerDecoderLayer(EMBEDDING_DIM, UNITS, 8)  # 创建Transformer解码器层

cnn_model = CNN_Encoder()  # 创建CNN模型
caption_model = ImageCaptioningModel(
    cnn_model=cnn_model, encoder=encoder, decoder=decoder, image_aug=image_augmentation,
)  # 创建图像字幕生成模型

在这段代码中，首先创建了Transformer编码器层encoder和Transformer解码器层decoder，使用了预定义的EMBEDDING_DIM和UNITS参数。

然后，创建了CNN模型cnn_model，使用了预定义的CNN_Encoder函数。

最后，通过ImageCaptioningModel类创建了图像字幕生成模型caption_model，将CNN模型、编码器和解码器作为参数传递进去，并指定了图像增强器image_augmentation。

这个模型可以用于训练和生成图像字幕。

In [ ]:
cross_entropy = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=False, reduction="none"
)  # 使用稀疏分类交叉熵作为损失函数，不计算logits，返回未经缩减的损失

early_stopping = tf.keras.callbacks.EarlyStopping(
    patience=3, restore_best_weights=True
)  # 提前停止训练的回调函数，如果连续3个epoch没有改善，恢复到最佳权重状态

caption_model.compile(
    optimizer=tf.keras.optimizers.Adam(),  # 使用Adam优化器
    loss=cross_entropy  # 损失函数使用交叉熵
)  # 编译图像字幕生成模型

在这段代码中，首先创建了稀疏分类交叉熵损失函数cross_entropy，其中from_logits=False表示输入的预测结果经过了softmax，reduction="none"表示不进行损失的缩减。

然后，创建了EarlyStopping回调函数early_stopping，设置了patience=3表示连续3个epoch没有改善时停止训练，并使用restore_best_weights=True恢复到最佳权重状态。

最后，通过compile方法，配置了图像字幕生成模型caption_model的优化器为Adam优化器，并将损失函数设置为前面定义的交叉熵损失函数cross_entropy。

这样，模型就准备好进行训练了

In [ ]:
history = caption_model.fit(
    train_dataset,  # 训练数据集
    epochs=5,  # 迭代次数
    validation_data=val_dataset,  # 验证数据集
    callbacks=[early_stopping]  # 回调函数，提前停止训练
)

通过调用fit方法，开始对图像字幕生成模型进行训练。传入训练数据集train_dataset作为训练数据，设置迭代次数为5次epochs=5，传入验证数据集val_dataset作为验证数据用于评估模型性能。还传入了提前停止训练的回调函数callbacks=[early_stopping]，以便在连续3个epoch没有改善时提前停止训练，并恢复到最佳权重状态。

训练过程中的训练损失、验证损失以及其他指标可以通过history对象来访问，如history.history['loss']可以获取训练过程中的训练损失历史记录。

In [ ]:
idx2word(2).numpy().decode('utf-8')
#要使用idx2word进行索引查找，您需要传递一个整数索引作为输入，并使用.numpy()方法将结果转换为NumPy数组。
#然后，您可以使用.decode('utf-8')将NumPy数组转换为UTF-8编码的字符串。

In [ ]:
def load_image_from_path(img_path):
    img = tf.io.read_file(img_path)
    img = tf.io.decode_jpeg(img, channels=3)
    img = tf.keras.layers.Resizing(299, 299)(img)
    img = img / 255.
    return img


def generate_caption(img_path):
    img = load_image_from_path(img_path)
    img = tf.expand_dims(img, axis=0)
    img_embed = caption_model.cnn_model(img)
    img_encoded = caption_model.encoder(img_embed, training=False)

    y_inp = '[start]'
    for i in range(MAX_LENGTH-1):
        tokenized = tokenizer([y_inp])[:, :-1]
        mask = tf.cast(tokenized != 0, tf.int32)
        pred = caption_model.decoder(
            tokenized, img_encoded, training=False, mask=mask)
        
        pred_idx = np.argmax(pred[0, i, :])
        pred_word = idx2word(pred_idx).numpy().decode('utf-8')
        if pred_word == '[end]':
            break
        
        y_inp += ' ' + pred_word
    
    y_inp = y_inp.replace('[start] ', '')
    return y_inp


上面的代码定义了两个函数：

load_image_from_path(img_path): 从给定的图像路径加载图像，并进行预处理和归一化处理。该函数返回处理后的图像张量。

generate_caption(img_path): 生成图像的标题。首先，它会加载和预处理图像。然后，它使用caption_model中的CNN模型和Transformer模型编码图像，并根据已生成的部分标题预测下一个词。循环迭代直到生成的标题达到最大长度或预测的词为[end]。最后，返回生成的标题字符串。

您可以将所需的图像路径传递给generate_caption函数，它将返回生成的标题字符串。请确保您已经训练并加载了适当的模型，并且caption_model是已经实例化的ImageCaptioningModel模型。

In [ ]:
# 从验证集中随机选择一张图像
idx = random.randrange(0, len(val_imgs))
img_path = val_imgs[idx]

# 使用图像路径生成预测的标题
pred_caption = generate_caption(img_path)
print('Predicted Caption:', pred_caption)
print()

# 显示图像和生成的标题
image = Image.open(img_path)
plt.imshow(image)
plt.axis('off')
plt.show()

In [ ]:
import requests
from PIL import Image

# 下载图像并保存为临时文件
url = "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcT2j6yclbKYDav4BGUKLAdTvSFXp1gtuzy5DQ&usqp=CAU"
im = Image.open(requests.get(url, stream=True).raw)
im.save('tmp.jpg')

# 生成图像的标题
pred_caption = generate_caption('tmp.jpg')

# 打印生成的标题并显示图像
print('Predicted Caption:', pred_caption)
#print(im)